In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pickle

# Function to segment data
def segment_data(data, labels, window_size, step_size):
    num_timesteps, num_features = data.shape
    segmented_data = []
    segmented_labels = []
    
    for start in range(0, num_timesteps - window_size + 1, step_size):
        end = start + window_size
        segmented_data.append(data[start:end, :])
        
        # Assign the label based on the majority label in the window
        window_label = np.bincount(labels[start:end]).argmax()
        segmented_labels.append(window_label)
    
    return np.array(segmented_data), np.array(segmented_labels)

# Function to preprocess data and save it
def preprocess_and_save(data, labels, window_size=20, step_size=5, test_size=0.1, val_size=0.2, output_file='processed_data.pkl'):
    # Normalize all input columns
    scaler = StandardScaler()
    data_normalized = scaler.fit_transform(data)
    
    # Segment data
    segmented_data, segmented_labels = segment_data(data_normalized, labels, window_size, step_size)
    
    # Shuffle data
    indices = np.arange(segmented_data.shape[0])
    np.random.shuffle(indices)
    segmented_data = segmented_data[indices]
    segmented_labels = segmented_labels[indices]
    
    # Train, validation, and test split
    X_train, X_test, y_train, y_test = train_test_split(segmented_data, segmented_labels, test_size=test_size, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size, random_state=42)
    
    # Save processed data
    with open(output_file, 'wb') as f:
        pickle.dump({
            'X_train': X_train,
            'X_val': X_val,
            'X_test': X_test,
            'y_train': y_train,
            'y_val': y_val,
            'y_test': y_test,
            'scaler': scaler
        }, f)
    
    return output_file

# Main code to preprocess the provided dataset
if __name__ == '__main__':
    # Load the dataset
    file_path = 'Merged_dataset.xlsx'  # Update with the correct path to your file
    sheet_data = pd.ExcelFile(file_path).parse('Sheet1')

    # Drop the first row and rename columns for clarity
    cleaned_data = sheet_data.iloc[1:].copy()
    cleaned_data.columns = ['I1_q1', 'I1_q2', 'I1_q3', 'I1_q4', 'I1_q5', 'I1_R/S',
                            'I2_q1', 'I2_q2', 'I2_q3', 'I2_q4', 'I2_q5', 'I2_R/S',
                            'I3_q1', 'I3_q2', 'I3_q3', 'I3_q4', 'I3_q5', 'I3_R/S',
                            'Condition']

    # Select relevant columns (all except Condition for input, Condition for labels)
    signal_columns = [col for col in cleaned_data.columns if col != 'Condition']
    labels_column = 'Condition'
    selected_data = cleaned_data[signal_columns + [labels_column]].copy()

    # Convert data to numeric and drop NaN values
    selected_data = selected_data.apply(pd.to_numeric, errors='coerce')
    selected_data = selected_data.dropna()

    # Extract features and labels
    data_array = selected_data[signal_columns].values
    labels_array = selected_data[labels_column].values.astype(int)  # Ensure labels are integers

    # Preprocess and save the data
    output_file = preprocess_and_save(data_array, labels_array, window_size=50, step_size=25)
    print(f"Processed data saved to: {output_file}")


Processed data saved to: processed_data.pkl


In [2]:
import pickle

# Load the .pkl file
file_path = 'processed_data.pkl'  # Update with the path to your .pkl file
with open(file_path, 'rb') as f:
    data = pickle.load(f)

# Access the contents
X_train = data['X_train']
X_val = data['X_val']
X_test = data['X_test']
y_train = data['y_train']
y_val = data['y_val']
y_test = data['y_test']
scaler = data['scaler']

# Print shapes of the datasets to confirm
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"y_test shape: {y_test.shape}")


X_train shape: (229, 50, 18)
X_val shape: (58, 50, 18)
X_test shape: (32, 50, 18)
y_train shape: (229,)
y_val shape: (58,)
y_test shape: (32,)
